# *Teknoloogiaa*: Chatbot Basé sur GPT-2

Ce notebook présente une démarche complète pour créer un chatbot en anglais à partir du modèle pré-entraîné GPT-2. Il couvre l’ensemble du processus, du traitement des données jusqu’à l’interface utilisateur.

### Objectifs

- Charger et préparer les données textuelles pour l’entraînement.
- Affiner le modèle GPT-2 avec des données personnalisées.
- Développer une fonction de réponse automatique basée sur le modèle.
- Intégrer le chatbot dans une interface interactive avec *Gradio*.

### Données et source:
- Les données utilisées comme corpus pour le Chatbot sont des commentaires issus de posts sur Reddit. Ces posts sont en rapport avec la technologie (l'Intelligence Artificielle (IA), le Machine Learning..)

### Étapes principales

1. *Installation des bibliothèques nécessaires* (Transformers, Gradio, etc.)
2. *Chargement et prétraitement des fichiers textes*
3. *Fine-tuning* du modèle GPT-2 avec gestion des ressources GPU
4. *Création d’une fonction de génération de réponses*
5. *Évaluation du chatbot*
6. *Déploiement de l'interface Gradio pour tester le chatbot*

### Dépendances

- transformers : pour charger, configurer et entraîner le modèle GPT-2.
- torch : la bibliothèque PyTorch utilisée pour entraîner et manipuler les modèles de deep learning.
- pandas : pour charger et manipuler les fichiers de données textuelles au format tabulaire.
- gradio : pour créer une interface web interactive et tester le chatbot en direct.
- sklearn : pour certaines fonctions utilitaires comme la séparation des jeux de données.
- numpy : pour les opérations numériques et la gestion efficace des tableaux de données.

### Installation et Importation des packages et des bases

In [9]:
! pip install gradio hf_xet
import os
import time
import math
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm.notebook import tqdm, trange
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import glob
from IPython.display import Markdown, display, HTML
import pandas as pd

### Téléchargement et fine-tuning du modèle

In [10]:
# ===============================================================
# SECTION 1: VÉRIFICATION DU MATÉRIEL ET UTILITAIRES DE BASE
# ===============================================================

# Vérifier la disponibilité du GPU
def check_gpu():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9  # En GB
        print(f"GPU disponible: {gpu_name} ({gpu_memory:.2f} GB)")
        return device
    else:
        print("Aucun GPU détecté, utilisation du CPU")
        return torch.device("cpu")

# ===============================================================
# SECTION 2: CHARGEMENT ET PRÉPARATION DES DONNÉES
# ===============================================================

# Classe pour charger les documents txt depuis notre repertoire
class SimpleDirectoryReader:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        
    def load_data(self):
        documents = []
        for file_path in glob.glob(os.path.join(self.directory_path, "*.txt")):
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                text = file.read()
                documents.append(Document(text, extra_info={"source": file_path}))
        return documents

class Document:
    def __init__(self, text, extra_info=None):
        self.text = text
        self.extra_info = extra_info or {}

# ===============================================================
# SECTION 3: TÉLÉCHARGEMENT ET PRÉPARATION DU MODÈLE GPT-2
# ===============================================================

# Téléchargeons le modèle GPT-2 Medium et le tokenizer
def download_and_save_model():
    model_dir = "/kaggle/working/gpt2-medium"

    start_time = time.time()
    
    # Télécharger le tokenizer et le modèle
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
    # Définir le token de padding pour GPT-2
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
    # Mettre à jour la configuration du modèle pour reconnaître le pad_token
    model.config.pad_token_id = model.config.eos_token_id
    
    # Sauvegarder le modèle et le tokenizer
    os.makedirs(model_dir, exist_ok=True)
    tokenizer.save_pretrained(model_dir)
    model.save_pretrained(model_dir)
    
    elapsed_time = time.time() - start_time
    
    # Afficher un message de confirmation
    print(f"Temps écoulé: {elapsed_time:.2f} secondes")
    
    return tokenizer, model

# ===============================================================
# SECTION 4: PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT
# ===============================================================

# Classe de dataset personnalisée
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, truncation=True, padding="max_length", 
                                  max_length=max_length, return_tensors="pt")
        
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item
    
    def __len__(self):
        return len(self.encodings["input_ids"])

# Fonction pour diviser les textes en chunks
def chunk_text(text, chunk_size_limit=600, overlap=20):
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size_limit - overlap):
        chunk = " ".join(words[i:i + chunk_size_limit])
        chunks.append(chunk)
    return chunks

# ===============================================================
# SECTION 5: CONSTRUCTION DE L'INDEX POUR LA RECHERCHE SÉMANTIQUE
# ===============================================================

# Construction de l'index
def construct_index(directory_path):
    # Vérifier le GPU
    device = check_gpu()
    
    # Paramètres
    max_input_size = 512  # Limité pour GPT-2
    num_outputs = 256
    max_chunk_overlap = 20
    chunk_size_limit = 600
    
    # Télécharger et sauvegarder le modèle
    tokenizer, model = download_and_save_model()
    model.to(device)
    
    # Vérifier si des documents existent dans le répertoire
    document_files = glob.glob(os.path.join(directory_path, "*.txt"))
    if not document_files:
        print(f"Aucun fichier .txt trouvé dans {directory_path}. Création d'un exemple simple...")

    documents = SimpleDirectoryReader(directory_path).load_data()
    
    if not documents:
        raise ValueError(f"Aucun document n'a été chargé depuis {directory_path}")
    
    print(f"{len(documents)} document(s) chargé(s)")
    
    # Prétraiter les documents
    all_chunks = []
    document_embeddings = []
    print(f"Création des chunks et des embeddings...")

    
    # Barre de progression pour les chunks afin de suivre le traitement
    for doc_idx, doc in enumerate(tqdm(documents, desc="Documents", position=0)):
        chunks = chunk_text(doc.text, chunk_size_limit, max_chunk_overlap)
        all_chunks.extend(chunks)
        
        # Créer des embeddings simples avec le modèle GPT-2
        for chunk_idx, chunk in enumerate(tqdm(chunks, desc=f"Embeddings pour doc {doc_idx+1}/{len(documents)}", position=1, leave=False)):
            inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True, max_length=max_input_size).to(device)
            with torch.no_grad():
                outputs = model(**inputs, output_hidden_states=True)
                # Utiliser la moyenne de la dernière couche cachée comme embedding
                last_hidden_state = outputs.hidden_states[-1].mean(dim=1)
                document_embeddings.append(last_hidden_state.squeeze().cpu().numpy())
    
    # Convertir en array numpy pour faciliter la récupération
    document_embeddings = np.array(document_embeddings)
    
    # Sauvegarder les données nécessaires
    index_data = {
        "chunks": all_chunks,
        "embeddings": document_embeddings,
    }
    
    torch.save(index_data, "/kaggle/working/index.pt")
    
    print(f"Index créé et sauvegardé dans /kaggle/working/index.pt")
    
    return index_data

# ===============================================================
# SECTION 6: FINE-TUNING DU MODÈLE SUR LES DOCUMENTS
# ===============================================================

# Fonction pour fine-tuner le modèle sur les documents
def fine_tune_model(directory_path, epochs=3, batch_size=4, learning_rate=5e-5):
    # Vérifier le GPU
    device = check_gpu()
    
    # Télécharger le modèle
    tokenizer, model = download_and_save_model()
    
    # Charger les documents
    documents = SimpleDirectoryReader(directory_path).load_data()
    texts = [doc.text for doc in documents]
    
    if not texts:
        raise ValueError(f"Aucun document trouvé pour le fine-tuning dans {directory_path}")
    
    print(f"{len(texts)} texte(s) chargé(s) pour l'entraînement")
    
    # Créer le dataset
    dataset = TextDataset(texts, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Configurer l'optimiseur
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )
    
    # Déplacer le modèle sur GPU 
    model.to(device)
    
    # Boucle d'entraînement
    
    model.train()
    
    # Initialiser le meilleur loss pour sauvegarder le meilleur modèle
    best_loss = float('inf')
    
    for epoch in range(epochs):
        print(f"\n🔄 Epoch {epoch+1}/{epochs}")
        epoch_start_time = time.time()
        total_loss = 0
        
        progress_bar = tqdm(dataloader, desc=f"Training", position=0, 
                           bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}')
        
        for step, batch in enumerate(progress_bar):
            # Obtenir les entrées et les envoyer sur GPU
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Réinitialiser les gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            
            # Backward pass
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            # Calculer les métriques et mettre à jour la barre de progression
            avg_loss = total_loss / (step + 1)
            elapsed = time.time() - epoch_start_time
            time_per_step = elapsed / (step + 1)
            remaining_steps = len(dataloader) - (step + 1)
            eta = time_per_step * remaining_steps
            
            # Mettre à jour la barre de progression avec des informations détaillées
            progress_bar.set_postfix({
                'loss': f'{avg_loss:.4f}',
                'elapsed': f'{int(elapsed//60)}m {int(elapsed%60)}s',
                'ETA': f'{int(eta//60)}m {int(eta%60)}s',
                'lr': f'{scheduler.get_last_lr()[0]:.2e}'
            })
            
            # Libérer la mémoire
            if step % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        # Calculer la perte moyenne pour cette époque
        epoch_avg_loss = total_loss / len(dataloader)
        epoch_time = time.time() - epoch_start_time
        
        print(f"Epoch {epoch+1}/{epochs} terminée: Loss = {epoch_avg_loss:.4f}, Temps = {int(epoch_time//60)}m {int(epoch_time%60)}s")
        
        # Sauvegarder le meilleur modèle
        if epoch_avg_loss < best_loss:
            best_loss = epoch_avg_loss
            best_model_dir = "/kaggle/working/gpt2-medium-fine-tuned-best"
            os.makedirs(best_model_dir, exist_ok=True)
            model.save_pretrained(best_model_dir)
            tokenizer.save_pretrained(best_model_dir)
            print(f"Meilleur modèle sauvegardé (loss: {best_loss:.4f})")
    
    # Sauvegarder le modèle fine-tuné final
    fine_tuned_dir = "/kaggle/working/gpt2-medium-fine-tuned"
    os.makedirs(fine_tuned_dir, exist_ok=True)
    model.save_pretrained(fine_tuned_dir)
    tokenizer.save_pretrained(fine_tuned_dir)
    
    display(HTML(f"""
    <div style="padding: 10px; border-radius: 5px; background-color: #eafaf1; border-left: 5px solid #2ecc71;">
      <h3 style="margin: 0;">✅ Fine-tuning terminé avec succès!</h3>
      <p>Modèle final sauvegardé dans <code>{fine_tuned_dir}</code></p>
      <p>Meilleur modèle sauvegardé dans <code>/kaggle/working/gpt2-medium-fine-tuned-best</code> (loss: {best_loss:.4f})</p>
    </div>
    """))
    
    return model, tokenizer

### Création de la fonction pour poser les questions

In [14]:
# Fonction pour poser des questions
def ask_me_anything(question, use_fine_tuned=True):
    # Charger le modèle approprié
    if use_fine_tuned and os.path.exists("/kaggle/input/base-de-reddit/modele-pre-entraine/modele-pre-entraine"):
        model_path = "/kaggle/input/base-de-reddit/modele-pre-entraine/modele-pre-entraine"
    else:
        model_path = "/kaggle/working/gpt2-medium"
    
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    # Définir le token de padding
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained(model_path)
    model.config.pad_token_id = model.config.eos_token_id
    
    # Charger l'index
    index_data = torch.load("/kaggle/input/base-de-reddit/index.pt")
    chunks = index_data["chunks"]
    embeddings = index_data["embeddings"]
    
    # Créer un embedding pour la question
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        question_embedding = outputs.hidden_states[-1].mean(dim=1).squeeze().numpy()
    
    # Calculer les similarités avec les chunks
    similarities = cosine_similarity([question_embedding], embeddings)[0]
    
    # Trouver les chunks les plus pertinents
    top_idx = np.argsort(similarities)[-3:][::-1]  # Top 3 chunks les plus similaires
    
    # CORRECTION: Limiter la taille du contexte pour ne pas dépasser max_length
    context_chunks = [chunks[i] for i in top_idx]
    
    # Choisir un contexte plus court si nécessaire
    prompt = f"Question: {question}\nContexte: {context_chunks[0]}\nRéponse:"
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Si le prompt est encore trop long, utiliser uniquement la question
    if inputs["input_ids"].shape[1] > 400:  # Laisse de la marge pour la génération
        prompt = f"Question: {question}\nRéponse:"
        inputs = tokenizer(prompt, return_tensors="pt")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    

    with torch.no_grad():
        output_sequences = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=200,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=1,
        )
    
    response = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    response = response.split("Réponse:")[-1].strip()
    
    display(Markdown(f"You asked: <b>{question}</b>"))
    display(Markdown(f"Bot says: <b>{response}</b>"))
    
    return response

### Entrainement du modèle sur nos données

In [4]:
# 1. Construire l'index
#index_data = construct_index("/kaggle/input/base-de-reddit/")

# 2. Fine-tuner le modèle
#model, tokenizer = fine_tune_model("/kaggle/input/base-de-reddit", epochs=3)

GPU disponible: Tesla T4 (15.83 GB)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch 1/3 terminée: Loss = 3.7245, Temps = 0m 1s
Meilleur modèle sauvegardé (loss: 3.7245)

🔄 Epoch 2/3


Training:   0%|                              | 0/1 [00:00<?, ?it/s]

Epoch 2/3 terminée: Loss = 3.2840, Temps = 0m 0s
Meilleur modèle sauvegardé (loss: 3.2840)

🔄 Epoch 3/3


Training:   0%|                              | 0/1 [00:00<?, ?it/s]

Epoch 3/3 terminée: Loss = 2.9781, Temps = 0m 0s
Meilleur modèle sauvegardé (loss: 2.9781)


### Tester le chatbot

In [15]:
ask_me_anything("What do people think about AI ?", use_fine_tuned=True)

/tmp/ipykernel_31/1451110233.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  index_data = torch.load("/kaggle/input/base-de-reddit/index.pt")
Setting `pad_token_id` to 

You asked: <b>What do people think about AI ?</b>

Bot says: <b>We are getting to the point where AI is not only going to be pervasive, it's going to be dominant. I would argue that by 2030, AI will have become a dominant force in our economy.
AI will be the norm and you'll see it everywhere. You'll see people doing things in their home, your car, your home office, your job, your home office. You'll see it everywhere.
AI is going to be ubiquitous and its impact will be felt across all sectors of our economy.
And by 2030, you'll see AI becoming more and more of a driver of economic growth.
The point is that AI is going to be a dominant force in the future.
AI is going to be ubiquitous and its impact will be felt across all sectors of our economy.
If AI is going to be pervasive, it's going to be dominant in all sectors.
If AI is going to be dominant in all sectors, it's going to be dominant in every</b>

"We are getting to the point where AI is not only going to be pervasive, it's going to be dominant. I would argue that by 2030, AI will have become a dominant force in our economy.\nAI will be the norm and you'll see it everywhere. You'll see people doing things in their home, your car, your home office, your job, your home office. You'll see it everywhere.\nAI is going to be ubiquitous and its impact will be felt across all sectors of our economy.\nAnd by 2030, you'll see AI becoming more and more of a driver of economic growth.\nThe point is that AI is going to be a dominant force in the future.\nAI is going to be ubiquitous and its impact will be felt across all sectors of our economy.\nIf AI is going to be pervasive, it's going to be dominant in all sectors.\nIf AI is going to be dominant in all sectors, it's going to be dominant in every"

In [6]:
#!zip -r /kaggle/working/gpt2-medium-fine-tuned-best.zip /kaggle/working/gpt2-medium-fine-tuned-best

  adding: kaggle/working/gpt2-medium-fine-tuned-best/ (stored 0%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/vocab.json (deflated 68%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/special_tokens_map.json (deflated 74%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/config.json (deflated 53%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/generation_config.json (deflated 24%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/merges.txt (deflated 53%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/model.safetensors (deflated 7%)
  adding: kaggle/working/gpt2-medium-fine-tuned-best/tokenizer_config.json (deflated 56%)


### Création de l'interface gradio

In [13]:
import gradio as gr
import torch
import os

# Définir la variable device si elle n'est pas déjà définie
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Fonction pour interagir avec le chatbot via Gradio
def gradio_ask(question, history):
    # Chargement des modèles et des données
    if not os.path.exists("/kaggle/working/index.pt"):
        return "Erreur: Veuillez d'abord exécuter construct_index() pour créer l'index.", history
    
    # Vous pouvez modifier ce paramètre pour utiliser le modèle fine-tuné ou non
    use_fine_tuned = os.path.exists("/kaggle/working/gpt2-medium-fine-tuned")
    
    # Utiliser la fonction existante
    response = ask_me_anything(question, use_fine_tuned=use_fine_tuned)
    
    # Formater pour Gradio - histoire normale, pas de type 'messages'
    history.append((question, response))
    return "", history

# Création de l'interface Gradio
def create_gradio_interface():
    with gr.Blocks(title="Teknoloogiaa") as demo:
        gr.Markdown("# Teknoloogiaa")
        gr.Markdown("Posez vos questions a Teknoloogiaa sur tout ce qui est en rapport avec l'IA, le machine learning, la datascience, ...")
        
        # Retirez le paramètre type='messages'
        chatbot = gr.Chatbot(height=300)
        
        # Création d'une ligne avec un champ texte et un bouton d'envoi
        with gr.Row():
            msg = gr.Textbox(placeholder="Posez votre question ici...", lines=1, scale=4)
            submit_btn = gr.Button("Envoyer", scale=1)
        
        clear = gr.Button("Effacer la conversation")
        
        # Connecter le bouton d'envoi à la fonction gradio_ask
        submit_btn.click(gradio_ask, [msg, chatbot], [msg, chatbot])
        
        # Garder la possibilité d'envoyer en appuyant sur Entrée
        msg.submit(gradio_ask, [msg, chatbot], [msg, chatbot])
        
        clear.click(lambda: None, None, chatbot, queue=False)
        
        
    return demo

# Pour lancer l'interface:
demo = create_gradio_interface()
demo.launch()

/tmp/ipykernel_31/3760614080.py:31: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=300)


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://e238daadaf84a1a7e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
